In [ ]:
!pip install timm

In [ ]:
!pip install torchinfo==1.6.3

In [ ]:
!pip install sklearn

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, models
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from tqdm import tqdm
import os
from sklearn.metrics import accuracy_score
import timm
from tqdm import tqdm  
from torchinfo import summary
from torch.optim.lr_scheduler import CosineAnnealingLR
from ranger import Ranger


# Write necessary utils

In [ ]:
root_in = '/root/autodl-tmp/Dogs' #Folder with input (image, lable)
root_out = '/root/autodl-tmp/Dogs/Output' #Folder with output image 
have_index = False # If the breed label have been map to a index

In [ ]:
def data_pre_access(file, output):
    '''transfer train label into index'''
    labels = pd.read_csv(file, index_col='id')
    labels_map = dict()
    labels['label_index'] = torch.zeros((labels.shape[0])).type(torch.int32).numpy()
    for i, label in enumerate(labels.breed.unique()):
        labels_map[i] = label
        labels.loc[labels.breed == label, 'label_index'] = i
    labels.to_csv(output)
    
    return labels_map

In [ ]:
if have_index:
    labels_map = {}
    t_data = pd.read_csv(os.path.join(root_out,'labels_index.csv'))
    
    def label_f(m):
        labels_map[int(m.label_index)] = m.label
        
    t_data.apply(label_f,axis=1)
else:
    labels_map = data_pre_access(os.path.join(root_in,'labels.csv'), output=os.path.join(root_out,'labels_index.csv'))

In [ ]:
len(labels_map)

In [ ]:
def test_data_pre_access(direction, output):
    '''list the test image direction in a csv'''
    folder = os.listdir(direction)
    files = [file for file in folder if os.path.isfile(os.path.join(direction, file))]
    
    test_dataframe = pd.DataFrame({'id':files})
    
    def split_jpg(m):
        id = str(m.id).split('.jpg')
        s.append(id[0])
    s = []
    test_dataframe.apply(split_jpg, axis=1)
    test_dataframe['id'] = s
    
    test_dataframe.to_csv(os.path.join(output, 'test.csv'))
    return test_dataframe

In [ ]:
test_data = test_data_pre_access(os.path.join(root_in,'test'),root_out)
test_data

In [ ]:
'''test_people = test_data_pre_access(os.path.join('/root/autodl-tmp/people'),root_out)
test_people'''

In [ ]:
test_data.set_index('id')

In [ ]:
class Dogs_Train_Dataset(Dataset):
    '''Train Dataset'''
    def __init__(self, file_in, transform=None):
        self.img_paths = pd.read_csv(file_in)
        self.transform = transform
        
    def __len__(self):
        return self.img_paths.shape[0]
    
    def __getitem__(self, index):
        img = Image.open(os.path.join(root_in, 'train', self.img_paths.iloc[index, 0] + '.jpg'))
        label_index = self.img_paths.iloc[index, 2]
        if self.transform:
            img = self.transform(img)
        return img, label_index
    

In [ ]:
class Dogs_Test_Dataset(Dogs_Train_Dataset):
    '''Test Dataset'''
    def __getitem__(self, index):
        img = Image.open(os.path.join(root_in, 'test', self.img_paths.iloc[index, 1] + '.jpg'))
        if self.transform:
            img = self.transform(img)
        return img

In [ ]:
class Accumulator():
    '''A counter util, which count the float value of the input'''
    def __init__(self, nums):
        self.metric = list(torch.zeros((nums,)).numpy())
        
    def __getitem__(self, index):
        return self.metric[index]
    
    def add(self, *args):
        for i, item in enumerate(args):
            self.metric[i] += float(item)
    

In [ ]:
a = Accumulator(2)

In [ ]:
#a.add(0,0)
metric = a 
metric[0]

In [ ]:
def accuracy(y_hat, y):
    '''used to count the right type'''
    y_hat = y_hat.exp().argmax(dim=1)
    y_hat.reshape((-1))
    y.reshape((-1))
    return accuracy_score(y.cpu().numpy(), y_hat.cpu().numpy(), normalize=False)

In [ ]:
def evaluate_accuracy(net, data_iter, device=None):
    '''Evalue the valid dataset'''
    if isinstance(net, nn.Module):
        net.eval()
        if not device:
            device = next(iter(net.parameters())).device
    
    metric = Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y.to(device)
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [ ]:
def predict_test(net, test_iter, device=None):
    '''Inference'''
    if isinstance(net, nn.Module):
        net.eval()
        if not device:
            device = next(iter(net.parameters())).device
    y = []
    net.to(device)
    softmax = nn.Softmax(dim=1)
    with torch.no_grad():
        for X in test_iter:
            if isinstance(X, list):
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y += softmax(net(X).cpu())

    return list(Y.numpy() for Y in y)
    

# Load data

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(256),
    # 从图像中心裁切224x224大小的图片
    transforms.CenterCrop(224),
    # 随机裁剪图像，所得图像为原始面积的0.2到1之间，高宽比在3/4和4/3之间。
    # 然后，缩放图像以创建224 x 224的新图像
    transforms.RandomResizedCrop(224, scale=(0.2, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
    transforms.RandomHorizontalFlip(),
    # 随机更改亮度，对比度和饱和度
    #transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.ToTensor(),
    # 标准化图像的每个通道
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    # 从图像中心裁切224x224大小的图片
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [ ]:
dog_train_dataset = Dogs_Train_Dataset(os.path.join(root_out, 'labels_index.csv'),transform=train_transform)
dog_val_dataset = Dogs_Train_Dataset(os.path.join(root_out, 'labels_index.csv'),transform=val_test_transform)

In [ ]:
train_length = int(len(dog_train_dataset) * 0.90) # train_length is not used
valid_length = len(dog_train_dataset) - train_length # define the valid_dataset's length
_, dog_val_dataset = random_split(dog_val_dataset, [train_length, valid_length])

#dog_val_dataset.transform = val_test_transform
dog_test_dataset = Dogs_Test_Dataset(os.path.join(root_out, 'test.csv'),transform=val_test_transform)

In [ ]:
dog_train_loader = DataLoader(dog_train_dataset, batch_size=32, shuffle=True, drop_last=True, num_workers=20)
dog_val_loader = DataLoader(dog_val_dataset, batch_size=32, shuffle=True, num_workers=20)
dog_test_loader = DataLoader(dog_test_dataset, batch_size=16, shuffle=False, num_workers=20)

# Finetune Pretrained Model

In [ ]:
model = timm.create_model('resnest200e', pretrained=True, num_classes=len(labels_map))
summary(model,input_size = (32, 3, 224, 224))

In [ ]:
param_name = [name for name,_ in model.named_parameters()] # All parameters name
layer_name = [name for name,_ in model.named_modules()] # All layers name

In [ ]:
param_name[-12:], layer_name[-20:]
len(param_name)

In [ ]:
#model = nn.Sequential(model, nn.ReLU(), nn.Dropout(0.3), nn.Linear(len(labels_map), len(labels_map)))

### Load model state from the .params 

In [ ]:
model.load_state_dict(torch.load('/root/autodl-tmp/Dogs/Output/Dogs_Breed.params'))

In [ ]:
def freeze_pretrained_layers(model):
    '''Freeze all layers except the last layer(fc or classifier)'''
    for param in model.parameters():
            param.requires_grad = False
    #nn.init.xavier_normal_(model.fc.weight)
    #nn.init.zeros_(model.fc.bias)
    model.fc.weight.requires_grad = True
    model.fc.bias.requires_grad = True

In [ ]:
freeze_pretrained_layers(model)
model.fc.weight.requires_grad

In [ ]:
def debarcle_layers(model, num_debarcle):
    '''Debarcle From the last [-1]layer to the [-num_debarcle] layers, 
    approximately(for there is Conv2d which has only weight parameter)'''
    num_debarcle *= 2
    param_debarcle = param_name[-num_debarcle:]
    if param_debarcle[0].split('.')[-1] == 'bias':
        param_debarcle = param_name[-(num_debarcle + 1):]
    for name, param in model.named_parameters():
        param.requires_grad = True if name in param_debarcle else False

In [ ]:
debarcle_layers(model, 200) # Debarcle the last 200 layers()

In [ ]:
def train_model(net, num_epochs, train_loader, val_loader, loss, lr, device, lr_min=1e-5, weight_decay=0, optim='sgd', use_amp=False, init=True, scheduler_type='Cosine'):
    '''Parameters:
        lr(float): the begining learning rate
        lr_min(float): min learning rate
        optim(String): the optimizer type
        use_amp(Boolean): Use mixed precision on GPU or not
        init(Boolean): Need initial the layers parameter or not
        scheduler_type(String): Learning rate scheduler
        
       Detail:
        The train process will save the model's parameter every 10 epochs.
        Every epoch, scheduler update once, and evaluate the train_dataset's accuracy and print it to std 5 times, 
        print the valid_dataset's accuracy once.
        If the valid accuracy >= 0.90, save the model's parameters as well.
    '''
    def init_xavier(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_normal_(m.weight)
    
    if init:
        net.apply(init_xavier)
    
    print('training on', device)
    net.to(device)
    
    if optim == 'sgd':
        optimizer = torch.optim.SGD((param for param in net.parameters() if param.requires_grad), lr=lr, weight_decay=weight_decay)
    elif optim == 'adam':
        optimizer = torch.optim.Adam((param for param in net.parameters() if param.requires_grad), lr=lr, weight_decay=weight_decay)
    elif optim =='adamW':
        optimizer = torch.optim.AdamW((param for param in net.parameters() if param.requires_grad), lr=lr, weight_decay=weight_decay)
    elif optim == 'ranger':
        optimizer = Ranger((param for param in net.parameters() if param.requires_grad), lr=lr, weight_decay=weight_decay)
        
    Value_train_l = list()
    Value_train_acc = list()
    Value_test_acc = list()
    
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp) # mixed_precison
    
    if scheduler_type == 'Cosine':
        scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=lr_min)
    
    num_batches = len(train_loader)
    for epoch in range(num_epochs):
        net.train()
        metric = Accumulator(3)
        
        for i, (X, y) in enumerate(tqdm(train_loader)):
            
            X ,y = X.to(device), y.to(device)
            
            with torch.cuda.amp.autocast(enabled=use_amp):
                y_hat = net(X)
                l = loss(y_hat, y)
                
            scaler.scale(l).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            
            with torch.no_grad():
                metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
                
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                print(epoch + (i + 1) / num_batches,
                             'train_l train_acc\t',(train_l, train_acc,None))
                
                Value_train_l.append(train_l)
                Value_train_acc.append(train_acc)
                Value_test_acc.append(None)
            
        scheduler.step()

        test_acc = evaluate_accuracy(net, val_loader, device=device)
        print('lr = ', optimizer.param_groups[0]['lr'])
        print(epoch + 1,'test_acc\t', (None, None, test_acc))

        Value_train_l.append(None)
        Value_train_acc.append(None)
        Value_test_acc.append(test_acc)
        if epoch % 10 == 0 or test_acc >= 0.90:
            torch.save(net.state_dict(),os.path.join(root_out, 'Dogs_Breed_' + str(epoch + 1) + '.params'))
    
        record_data = pd.DataFrame(zip(Value_train_l, Value_train_acc, Value_test_acc))    
        record_data.to_csv(os.path.join(root_out, 'Record_Dogs_Breed_ResNest.csv')) 
        
    torch.save(net.state_dict(),os.path.join(root_out, 'Dogs_Breed.params'))
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'on {str(device)}')

In [ ]:
'''Maps the model's breed order into submission breed order'''
s = ''
with open('/root/autodl-tmp/Dogs/Output/breed.txt') as f:
    while(True):
        c = f.read()
        if c == '':
            break
        else:
            s += c
ls = s.split('\n')
ls = ls[0:len(ls)-1]

In [ ]:
#people_dataset = Dogs_Test_Dataset(os.path.join(root_out, 'test.csv'),transform=val_test_transform)
#people_loader = DataLoader(people_dataset, shuffle=False)

In [ ]:
labels_map.get(0)

In [ ]:
result_index

In [ ]:
lr, epochs, device = 3e-4, 30, 'cuda' if torch.cuda.is_available() else 'cpu'
lr_min = 1e-5
#train_model(model, epochs, dog_train_loader, dog_val_loader, nn.CrossEntropyLoss(), lr, device, lr_min=lr_min, optim='ranger', use_amp=True, init=False, scheduler_type='Cosine')

result_index = predict_test(model, dog_test_loader, device=device)
result = pd.read_csv(os.path.join(root_out, 'test.csv'))
for i in range(len(labels_map)):
    result[labels_map.get(i)] = [x[i] for x in result_index]
#result_breed = [labels_map.get(i) for i in result_index]
#result['breed'] = result_breed
a = pd.DataFrame(result)
result = result.set_index('id')[ls]
result.to_csv(os.path.join(root_out, 'submission.csv'))


In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_snapshot()

In [ ]:
torch.cuda.memory_stats()

In [ ]:
result.to_csv(os.path.join(root_out, 'submission.csv'))

In [ ]:
result

In [ ]:
a.set_index('id')

In [ ]:
s = [labels_map.get(i) for i in a]
s